In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())


Looking for [chromedriver 80.0.3987.106 mac64] driver in cache 
File found in cache by path [/Users/barry/.wdm/drivers/chromedriver/80.0.3987.106/mac64/chromedriver]


In [3]:
# 20 job postings per page
def get_urls(url):
    
    nums = 25
    # get html back
    html = requests.get(url)
    html = html.text
    soup = BeautifulSoup(html, 'lxml')

    # add the common part between all search pages
    base_url = 'https://www.coursera.org/search?query=artificial%20intelligence&indices%5Bprod_all_products_term_optimization%5D%5Bpage%5D=2&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BruleContexts%5D%5B0%5D=en&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
    urls = []
    urls.append(base_url)
    for i in range(2, nums):
        a = "https://www.coursera.org/search?query=artificial%20intelligence&indices%5Bprod_all_products_term_optimization%5D%5Bpage%5D="+str(i)+'&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BclickAnalytics%5D=true&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BruleContexts%5D%5B0%5D=en&indices%5Bprod_all_products_term_optimization%5D%5Bconfigure%5D%5BhitsPerPage%5D=10&configure%5BclickAnalytics%5D=true'
        urls.append(a)
        
    return urls

In [4]:
urls = get_urls('https://www.coursera.org/search?query=artificial%20intelligence&')

In [5]:
course_links, course_desc = [],[]
# loop over all page-urls
for page_url in urls:

    # get the HTML of the search results page
    page = requests.get(page_url)
    content = page.text
    soup = BeautifulSoup(content, 'lxml')

    results = soup.find_all('li',{'class':'ais-InfiniteHits-item'})

    for course in results:

        course_link = 'https://www.coursera.org' + course.find('a')['href']
        
        #description
        course = requests.get(course_link)
        course_soup = BeautifulSoup(course.text, 'lxml')
        try:
            desc = course_soup.find('div',{'class':'description'}).text
        except:
            desc = 'No Descriptions'
        course_desc.append(desc)
        
        course_links.append(course_link)

In [6]:
#chapter links
course_len,course_names,chapter_names,chapter_links = [],[],[],[]
for link in course_links:
    
    page = requests.get(link)
    content = page.text
    soup = BeautifulSoup(content, 'lxml')
    
    course_names.append(soup.find('h1').text)
    #click buttons and generate full_link to further operations
    driver.get(link)
    time.sleep(5)
    try:
        button = driver.find_element_by_xpath('//*[@id="main"]/div/div[6]/div/div/div[2]/ul/li/div[2]/button/span')
        button.click()
        all_link = driver.page_source
    except:
        all_link = driver.page_source
    
    course_soup = BeautifulSoup(all_link, 'lxml')
    
    #course chapters names
    chapters = course_soup.find_all('h3',{'class':"headline-3-text bold m-t-1 m-b-2"})
    for i in chapters:
        chapter_names.append(i.text)
    
    #chapter links
    a = course_soup.find_all('div',{'class':'Row_nvwp6p CourseItem'})
    course_len.append(len(a))
    for b in a:
        chapter_links.append('https://www.coursera.org' + b.find('a',{'data-e2e':"course-link"})['href'])

In [7]:
len(chapter_names)

221

In [8]:
len(chapter_links)

221

In [9]:
#section names and description
section_len = []

for link in chapter_links:
    driver.get(link)
    time.sleep(5)
    try:
        buttons = driver.find_element_by_xpath('//*[@id="expand_module_description_button"]')
        buttons.click()
        full_link = driver.page_source
    except:
        full_link = driver.page_source
        
    chapter_soup = BeautifulSoup(full_link, 'lxml')
    
    #names and len
    a = chapter_soup.find_all('h2',{'class':'headline-2-text bold m-b-2'})
    section_len.append(len(a))

In [10]:
section_names,section_desc = [],[]
for link in chapter_links:
    driver.get(link)
    time.sleep(5)
    try:
        buttons = driver.find_element_by_xpath('//*[@id="expand_module_description_button"]')
        buttons.click()
        full_link = driver.page_source
    except:
        full_link = driver.page_source
        
    chapter_soup = BeautifulSoup(full_link, 'lxml')
    
    a = chapter_soup.find_all('h2',{'class':'headline-2-text bold m-b-2'})
    
    #names
    for b in a:
        section_names.append(b.text)
    
    #desc
    c = chapter_soup.find_all('div',{'class':'content-inner'})[-len(a):]
    for d in c:
        section_desc.append(d.text)

In [14]:
len(section_names)

1072

In [11]:
len(section_desc)

1074

In [15]:
df_section = pd.DataFrame({'Section Names':section_names,'Section Descriptions':section_desc[:-2]})
df_section

,Section Names,Section Descriptions
0,Defining Data Science and What Data Scientists Do,"In this module, you will hear from Norman Whit..."
1,Data Science Topics,"In this module, you will learn about the appro..."
2,Data Science in Business,IBM offers a wide range of technology and cons...
3,Introducing Skills Network Labs,"This week, you will learn about a popular data..."
4,Jupyter Notebooks,"This week, you will learn about Apache Zeppeli..."
...,...,...
1067,Course Orientation,You will find out where to go next after compl...
1068,"Module 1: Spark, Hortonworks, HDFS, CAP",The University of Illinois at Urbana-Champaign...
1069,Module 2: Large Scale Data Storage,"In Module 1, we introduce you to the world of ..."
1070,Module 3: Streaming Systems,"In this module, you will learn about large sca..."


In [16]:
df_section.insert(1,'Section Length','')

In [17]:
df_section.insert(2,'Chapter Links','')

In [18]:
a = 0
for i in range(len(section_len)):
    num = section_len[i]
    link = chapter_links[i]
    df_section.iloc[a:a+num,1] = num
    df_section.iloc[a:a+num,2] = link
    a += num

In [19]:
df_section

,Section Names,Section Length,Chapter Links,Section Descriptions
0,Defining Data Science and What Data Scientists Do,3,https://www.coursera.org/learn/what-is-datasci...,"In this module, you will hear from Norman Whit..."
1,Data Science Topics,3,https://www.coursera.org/learn/what-is-datasci...,"In this module, you will learn about the appro..."
2,Data Science in Business,3,https://www.coursera.org/learn/what-is-datasci...,IBM offers a wide range of technology and cons...
3,Introducing Skills Network Labs,6,https://www.coursera.org/learn/open-source-too...,"This week, you will learn about a popular data..."
4,Jupyter Notebooks,6,https://www.coursera.org/learn/open-source-too...,"This week, you will learn about Apache Zeppeli..."
...,...,...,...,...
1067,Course Orientation,5,https://www.coursera.org/learn/cloud-applicati...,You will find out where to go next after compl...
1068,"Module 1: Spark, Hortonworks, HDFS, CAP",5,https://www.coursera.org/learn/cloud-applicati...,The University of Illinois at Urbana-Champaign...
1069,Module 2: Large Scale Data Storage,5,https://www.coursera.org/learn/cloud-applicati...,"In Module 1, we introduce you to the world of ..."
1070,Module 3: Streaming Systems,5,https://www.coursera.org/learn/cloud-applicati...,"In this module, you will learn about large sca..."


In [20]:
df_course = pd.DataFrame({'Course Names':course_names,'Course Length':course_len,'Course Descriptions':course_desc})

In [21]:
df_chapter = pd.DataFrame({'Chapter Names':chapter_names,'Chapter Links':chapter_links})

In [22]:
df_chapter.insert(0,'Course Names','')
df_chapter.insert(1,'Course Length','')
df_chapter.insert(2,'Course Descriptions','')

In [23]:
a = 0
for i in range(len(df_course)):
    num = df_course.iloc[i,1]
    df_chapter.iloc[a:a+num,0] = df_course.iloc[i,0]
    df_chapter.iloc[a:a+num,1] = df_course.iloc[i,1]
    df_chapter.iloc[a:a+num,2] = df_course.iloc[i,2]
    a += num

In [24]:
df_chapter

,Course Names,Course Length,Course Descriptions,Chapter Names,Chapter Links
0,IBM Data Science Professional Certificate,9,Data Science has been ranked as one of the hot...,What is Data Science?,https://www.coursera.org/learn/what-is-datasci...
1,IBM Data Science Professional Certificate,9,Data Science has been ranked as one of the hot...,Open Source tools for Data Science,https://www.coursera.org/learn/open-source-too...
2,IBM Data Science Professional Certificate,9,Data Science has been ranked as one of the hot...,Data Science Methodology,https://www.coursera.org/learn/data-science-me...
3,IBM Data Science Professional Certificate,9,Data Science has been ranked as one of the hot...,Python for Data Science and AI,https://www.coursera.org/learn/python-for-appl...
4,IBM Data Science Professional Certificate,9,Data Science has been ranked as one of the hot...,Databases and SQL for Data Science,https://www.coursera.org/learn/sql-data-science
...,...,...,...,...,...
216,Excel to MySQL: Analytic Techniques for Busine...,4,"Formulate data questions, explore and visualiz...",Managing Big Data with MySQL,https://www.coursera.org/learn/analytics-mysql...
217,Cloud Computing Specialization,4,The Cloud Computing Specialization takes you o...,"Cloud Computing Concepts, Part 1",https://www.coursera.org/learn/cloud-computing...
218,Cloud Computing Specialization,4,The Cloud Computing Specialization takes you o...,Cloud Computing Concepts: Part 2,https://www.coursera.org/learn/cloud-computing...
219,Cloud Computing Specialization,4,The Cloud Computing Specialization takes you o...,"Cloud Computing Applications, Part 1: Cloud Sy...",https://www.coursera.org/learn/cloud-applicati...


In [29]:
df_section.to_csv('section_topics_ai.csv')

In [30]:
df_chapter.to_csv('chapter_topics_ai.csv')